# Uploading the file

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving DataNeuron_Text_Similarity.csv to DataNeuron_Text_Similarity.csv


# Importing the required libraries

In [ ]:
import pandas as pd
import io
from nltk.corpus import wordnet
import autocorrect
from autocorrect import Speller
import re



In [ ]:
df = pd.read_csv(io.BytesIO(uploaded["DataNeuron_Text_Similarity.csv"]))
df

,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...
...,...,...
2995,uk directors guild nominees named martin scors...,steel firm to cut 45 000 jobs mittal steel ...
2996,u2 to play at grammy awards show irish rock ba...,israel looks to us for bank chief israel has a...
2997,pountney handed ban and fine northampton coach...,india and iran in gas export deal india has si...
2998,belle named best scottish band belle & sebas...,mido makes third apology ahmed mido hossam h...


# Preprocssing the file

In [ ]:
# Data preprocessing
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]


    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

In [ ]:
#Importing Required Libraries
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Transforming the data based on above pre-processing steps

In [ ]:
df['text1_processed'] = df['text1'].apply(preprocess_text)
df['text2_processed'] = df['text2'].apply(preprocess_text)

In [ ]:
df

In [ ]:
!pip install sentence-transformers

# Applying the SentenceTransformer model

In [ ]:
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2') #all-MiniLM-L12-v2 #all-MiniLM-L12-v2 #all-MiniLM-L6-v2
#model = SentenceTransformer('all-MiniLM-L12-v2')
# Encode sentences into embeddings
embeddings1 = model.encode(df['text1_processed'], convert_to_tensor=True)
embeddings2 = model.encode(df['text2_processed'], convert_to_tensor=True)

# Calculate the cosine similarity between the embeddings
similarity_scores = util.cos_sim(embeddings1, embeddings2)
average_similarity = similarity_scores.cpu().diagonal().numpy()
df['similarity'] = average_similarity

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Print the similarity score
print("Similarity Score:", average_similarity.item())

ValueError: can only convert an array of size 1 to a Python scalar

# Cheaking the DataFrame

In [ ]:
df

# Fine Tuning the model

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

# Load the SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Define your training examples
train_examples = [InputExample(texts=[text1, text2], label=similarity_score) for text1, text2, similarity_score in zip(df['text1_processed'], df['text2_processed'], df['similarity'])]

# Define your training dataloader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# Define your loss function
train_loss = losses.CosineSimilarityLoss(model)

# Fine-tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)



Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/188 [00:00<?, ?it/s]

In [ ]:
df

,text1,text2,text1_processed,text2_processed,similarity
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,broadband challenge tv viewing number european...,gardener win double glasgow britain jason gard...,0.187735
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,rap bos arrested drug find rap mogul marion su...,amnesty chief lament war failure lack public o...,0.167550
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,player burnout worry robinson england coach an...,hank greeted wintry premiere hollywood star to...,0.268997
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,heart oak cotonsport heart oak set ghanaian co...,redford vision sundance despite sporting cordu...,0.179239
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,sir paul rock super bowl crowd sir paul mccart...,mauresmo open victory la amelie mauresmo maria...,0.234295
...,...,...,...,...,...
2995,uk directors guild nominees named martin scors...,steel firm to cut 45 000 jobs mittal steel ...,uk director guild nominee named martin scorses...,steel firm cut job mittal steel one world larg...,0.188734
2996,u2 to play at grammy awards show irish rock ba...,israel looks to us for bank chief israel has a...,u play grammy award show irish rock band u pla...,israel look u bank chief israel asked u banker...,0.315396
2997,pountney handed ban and fine northampton coach...,india and iran in gas export deal india has si...,pountney handed ban fine northampton coach bud...,india iran gas export deal india signed bn bn ...,0.126793
2998,belle named best scottish band belle & sebas...,mido makes third apology ahmed mido hossam h...,belle named best scottish band belle sebastian...,mido make third apology ahmed mido hossam made...,0.222682


In [ ]:
file_path = "OutputPartA.csv"

# Convert the DataFrame to a CSV file
df.to_csv(file_path, index=False)

In [ ]:
# Manually Checking the model
text1 = "my name is himnashu , i have completed PG diploma"
text2 = "i have completed PG diploma in Big data analytics"
# Preprocess the texts
text1_processed = preprocess_text(text1)
text2_processed = preprocess_text(text2)

# Convert the texts to embeddings
embedding1 = model.encode([text1_processed], convert_to_tensor=True)
embedding2 = model.encode([text2_processed], convert_to_tensor=True)

# Calculate the cosine similarity
similarity_score = util.cos_sim(embedding1, embedding2)

# Print the similarity score
#print("Similarity Score:", similarity_score.item())

# Saving the model


In [ ]:
#model = SentenceTransformer("my_model")
model.save("my_models")


In [ ]:
from google.colab import files
files.download('my_models')